# Get All Posts' Details

In [153]:
from bs4 import BeautifulSoup

# Load the HTML file
with open(r'D:\projects\jekyllCM\kucmc_website\tools\FBpageOrg.html', 'r') as file:
    html_content = file.read()

# Parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

post_containers = soup.find_all('div', class_='x1cy8zhl x78zum5 x1q0g3np xod5an3 x1pi30zi x1swvt13 xz9dl7a')
print(len(post_containers))
# Extract post content
Allposts =[]
for container in post_containers:
    try:
        actualPostCont = container.parent.parent.parent.parent.parent
        postDetail={}

        post_text = actualPostCont.find_all('div',class_="xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs x126k92a")
        captionText=(" ".join([i.strip() for i in post_text[0].text.split("\n")]))
        print(captionText)
        postDetail["caption"]=captionText

        urlDateTag = actualPostCont.find("span",class_="x1rg5ohu x6ikm8r x10wlt62 x16dsc37 xt0b8zv")
        date = soup.find("text",id=urlDateTag.find("use").get("xlink:href")[1:]).text.strip()
        print(date)
        postDetail["date"]=date

        specific_href="https://www.facebook.com/photo/?fbid"
        images = actualPostCont.select(f'a[href*="{specific_href}"]')
        imgLinks =[]
        for a in images:
            # print(a.get("href"))
            imgLinks.append(a.find("img").get("src"))
            # print(a.find("img").get("src"))
        postDetail["imgs"]=imgLinks

        specific_href="comment_id"
        comments=actualPostCont.select(f'a[href*="{specific_href}"]')
        for comment in comments:
            url = (comment.get("href"))
            
            if str(url).find("https://www.facebook.com/kucmc1/posts/pfbid0") !=-1:
                url = url[:url.find("?comment_id")]
                if url.find("https://www.facebook.com/kucmc1/posts/pfbid02Q9uZyDkFYtTbZdkx8Tj4SXZ2t9xj6mhqGfxreHfWhqxKGFtEfUWFJX17ipozTAWql?comment_id=775008607335488&reply_comment_id=773449054223807"):
                    mainUrl="#"
                    continue
                mainUrl = url
                print(" s ",url)
                
        postDetail["url"]= mainUrl if mainUrl else '#'
        print(postDetail["url"])
        print(len(postDetail["imgs"]))
        # print(postDetail)
        print()
        if imgLinks:
            Allposts.append(postDetail)
    except:
        print("---")
    

96
  Acknowledging Prof. Dr. Sudarshan Tiwari from the Department of Mathematics at RPTU Kaiserslautern-Landau, Germany who delivered a talk titled "Disease Contagion Models Coupled to Crowd Motion and Mesh-Free Simulations."  
---
  Two of our students from CM third year participated in the hackathon Field Mapping Task Splitting Algorithm held in Islington college, Kathmandu and won the title for MOST CREATIVE IDEA .  
May 23 at 9:42 AM
#
0

  Join us for the finale of our 'Monthly Career with Alumni' series! It's been an incredible journey, and tonight marks the last interactive session. We're calling on all of you to be an active part of this event, happening today at 8:30 PM! Share your questions, engage with our amazing alumni, and gain valuable insights into shaping your own career path.  
May 14
#
1

  Workshop on "Monthly Career with Alumni" for May 7(today) will be held from 8:30 pm. We hope for your active participation !  
May 7
#
1

  Integration Bee for the first time in K

# Valid Date Maker

In [167]:
from datetime import datetime

def convert_dates(date_strings):
    current_year = datetime.now().year
    try:
        date_obj = datetime.strptime(date_strings, "%B %d, %Y")
    except ValueError:
        date_obj = datetime.strptime(date_strings, "%B %d")
        date_obj = date_obj.replace(year=current_year)
    return date_obj.strftime("%Y-%m-%d")

# Filename Validator

In [168]:
import re

def sanitize_folder_name(name):
    # Remove any characters that are not letters, numbers, or spaces
    sanitized_name = re.sub(r'[^\w\s]', '', name)
    # Remove leading/trailing spaces and replace internal spaces with underscores
    sanitized_name = sanitized_name.strip().replace(' ', '_')
    return sanitized_name


# Title Maker

In [169]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

def generate_title_from_paragraph(paragraph):
    # Initialize the parser and tokenizer
    parser = PlaintextParser.from_string(paragraph, Tokenizer("english"))

    # Initialize the LexRank summarizer
    summarizer = LexRankSummarizer()

    # Generate the summary (title) from the paragraph
    summary = summarizer(parser.document, 1)

    if summary:
        # Extract the title as a complete sentence
        title = str(summary[0]).strip()
    else:
        # Fallback: Use the first sentence as the title
        sentences = list(parser.document.sentences)
        if sentences:
            title = str(sentences[0]).strip()
        else:
            # If no sentences are present, return an empty string
            title = paragraph

    return title


# Keywords maker

In [170]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import string

nltk.download('punkt')
nltk.download('stopwords')

def extract_top_keywords(paragraph, top_n=5):
    # Tokenize the paragraph into words
    words = word_tokenize(paragraph)

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    words = [word.lower() for word in words if word.lower() not in stop_words and word not in punctuation]

    # Calculate word frequency distribution
    fdist = FreqDist(words)

    # Get the most common words as keywords
    top_keywords = fdist.most_common(top_n)

    return [keyword for keyword, _ in top_keywords]

[nltk_data] Downloading package punkt to C:\Users\jasw.LAPTOP-
[nltk_data]     SR8QA6EL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\jasw.LAPTOP-
[nltk_data]     SR8QA6EL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# File Creator

In [171]:
print(len(Allposts))
for post in Allposts:
    print(post["date"])
    date = convert_dates(post["date"])
    url = post["url"]
    caption =post["caption"]
    pictures=post["imgs"]
    title = generate_title_from_paragraph(caption)
    keywords= extract_top_keywords(caption)
    filename=date+"-"+"-".join(keywords)+".md"
    print(filename)

    mdContent=f"""---
layout: post
title: {title}
description: {caption}
picture: {pictures[0]}
labelclasses: ["label-default","label-primary","label-success","label-info","label-warning","label-danger"]
keywords: {keywords}
type: "post"
author: "FB"

pictures: {pictures}
---
{caption}<br>[Read More]({url})"""
    mdContent+='{:target="_blank"}{:rel="noopener noreferrer"}'
    print(mdContent)
    
    for pic in pictures:
        print(pic)
    filename = "D:\projects\jekyllCM\kucmc_website\_posts\\"+filename
    with open(filename,"w") as file:
        file.write(mdContent)
    
    

43
May 14
2023-05-14-career-alumni-join-us-finale.md
---
layout: post
title: Join us for the finale of our 'Monthly Career with Alumni' series!
description:   Join us for the finale of our 'Monthly Career with Alumni' series! It's been an incredible journey, and tonight marks the last interactive session. We're calling on all of you to be an active part of this event, happening today at 8:30 PM! Share your questions, engage with our amazing alumni, and gain valuable insights into shaping your own career path.  
picture: https://scontent.fktm20-1.fna.fbcdn.net/v/t39.30808-6/346467560_805001767860593_1583571056151436347_n.jpg?stp=dst-jpg_s960x960&_nc_cat=103&cb=99be929b-59f725be&ccb=1-7&_nc_sid=730e14&_nc_ohc=QN-m3II2LkkAX9uKtdw&_nc_ht=scontent.fktm20-1.fna&oh=00_AfC3J3E-JR7ivZuwmApQ1EmGhEp3uL4q3glW_uPZ4DwUkQ&oe=6490C0DC
labelclasses: ["label-default","label-primary","label-success","label-info","label-warning","label-danger"]
keywords: ['career', 'alumni', 'join', 'us', 'finale']
type: 